In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal, ndimage
from scipy.stats import norm
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.manifold import TSNE
import io
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10
plt.rcParams['image.cmap'] = 'gray'

print("="*80)
print("VISIÓN ARTIFICIAL - EJEMPLOS APLICADOS A IMÁGENES 2D")
print("="*80)

# ==============================================================================
# CARGAR IMAGEN
# ==============================================================================

print("\n¿Deseas subir tu propia imagen o usar una imagen sintética?")
print("Opciones:")
print("  1. Subir mi propia imagen")
print("  2. Usar imagen sintética generada automáticamente")

try:
    from google.colab import files
    EN_COLAB = True
    print("\n✓ Entorno Google Colab detectado")
except ImportError:
    EN_COLAB = False
    print("\n⚠ No se detectó Google Colab. Se usará imagen sintética por defecto.")

usar_imagen_propia = False

if EN_COLAB:
    opcion = input("\nIngresa tu opción (1 o 2): ").strip()
    
    if opcion == "1":
        print("\n📤 Por favor, sube tu imagen (formatos: JPG, PNG, BMP, etc.)")
        uploaded = files.upload()
        
        if uploaded:
            filename = list(uploaded.keys())[0]
            print(f"\n✓ Imagen cargada: {filename}")
            
            imagen_original = Image.open(io.BytesIO(uploaded[filename]))
            
            if imagen_original.mode != 'L':
                print("  Convirtiendo a escala de grises...")
                imagen_original = imagen_original.convert('L')
            
            imagen_original = np.array(imagen_original)
            
            max_size = 512
            if imagen_original.shape[0] > max_size or imagen_original.shape[1] > max_size:
                print(f"  Redimensionando imagen a máximo {max_size}x{max_size}...")
                ratio = min(max_size / imagen_original.shape[0], max_size / imagen_original.shape[1])
                new_size = (int(imagen_original.shape[1] * ratio), int(imagen_original.shape[0] * ratio))
                img_pil = Image.fromarray(imagen_original)
                img_pil = img_pil.resize(new_size, Image.BILINEAR)
                imagen_original = np.array(img_pil)
            
            usar_imagen_propia = True
            print(f"✓ Imagen lista: {imagen_original.shape}")
        else:
            print("\n⚠ No se subió ninguna imagen. Usando imagen sintética.")

if not usar_imagen_propia:
    print("\n[Generando imagen sintética...]")
    size = 512
    x = np.linspace(0, 4*np.pi, size)
    y = np.linspace(0, 4*np.pi, size)
    X, Y = np.meshgrid(x, y)
    
    imagen_original = (np.sin(X) + np.sin(Y) + 
                       0.5*np.sin(3*X) + 0.5*np.sin(3*Y) + 
                       0.3*np.sin(5*X + 5*Y))
    
    imagen_original = ((imagen_original - imagen_original.min()) / 
                       (imagen_original.max() - imagen_original.min()) * 255).astype(np.uint8)
    
    print(f"✓ Imagen sintética creada: {imagen_original.shape}")

img = imagen_original.astype(float) / 255.0

print(f"\nInformación de la imagen:")
print(f"  Dimensiones: {imagen_original.shape}")
print(f"  Rango de valores: [{imagen_original.min()}, {imagen_original.max()}]")

# Mostrar imagen original
plt.figure(figsize=(8, 8))
plt.imshow(imagen_original, cmap='gray')
titulo = 'Imagen Original' if usar_imagen_propia else 'Imagen Original (Sintética)'
plt.title(f'{titulo}\n{imagen_original.shape[0]}x{imagen_original.shape[1]} píxeles', 
          fontsize=14, fontweight='bold')
plt.colorbar(label='Intensidad')
plt.axis('off')
plt.tight_layout()
plt.show()

# ==============================================================================
# EJEMPLO 1: DETECCIÓN DE ANOMALÍAS EN IMÁGENES
# ==============================================================================

print("\n" + "="*80)
print("EJEMPLO 1: Detección de Anomalías en Imágenes usando Características Locales")
print("="*80)

def extract_local_features(image, patch_size=16):
    """
    Extrae características locales de parches de la imagen.
    
    Parámetros:
    - image: Imagen de entrada (normalizada 0-1)
    - patch_size: Tamaño del parche
    
    Retorna:
    - features: Matriz de características (n_patches, n_features)
    - positions: Posiciones de los parches
    """
    h, w = image.shape
    features = []
    positions = []
    
    for i in range(0, h - patch_size, patch_size // 2):
        for j in range(0, w - patch_size, patch_size // 2):
            patch = image[i:i+patch_size, j:j+patch_size]
            
            if patch.shape == (patch_size, patch_size):
                # Características estadísticas
                feat = [
                    np.mean(patch),           # Media
                    np.std(patch),            # Desviación estándar
                    np.median(patch),         # Mediana
                    np.max(patch) - np.min(patch),  # Rango
                    np.percentile(patch, 25), # Cuartil 1
                    np.percentile(patch, 75), # Cuartil 3
                ]
                
                # Gradientes
                gy, gx = np.gradient(patch)
                feat.extend([
                    np.mean(np.abs(gx)),      # Gradiente X medio
                    np.mean(np.abs(gy)),      # Gradiente Y medio
                    np.sqrt(np.mean(gx**2 + gy**2))  # Magnitud del gradiente
                ])
                
                features.append(feat)
                positions.append((i + patch_size//2, j + patch_size//2))
    
    return np.array(features), np.array(positions)

print("\n[1.1] Extrayendo características locales de la imagen...")
features, positions = extract_local_features(img, patch_size=16)
print(f"✓ Extraídas {len(features)} características de parches")

# Entrenar SVM de una clase
print("\n[1.2] Entrenando detector de anomalías (One-Class SVM)...")
svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
svm.fit(features)

# Predecir anomalías
predictions = svm.predict(features)
scores = svm.decision_function(features)

n_anomalies = np.sum(predictions == -1)
print(f"✓ Anomalías detectadas: {n_anomalies} de {len(features)} parches ({n_anomalies/len(features)*100:.1f}%)")

# Crear mapa de anomalías
anomaly_map = np.zeros_like(img)
score_map = np.zeros_like(img)

for (y, x), pred, score in zip(positions, predictions, scores):
    if pred == -1:
        anomaly_map[max(0, y-8):min(img.shape[0], y+8), 
                    max(0, x-8):min(img.shape[1], x+8)] = 1
    score_map[y, x] = score

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Imagen Original', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(anomaly_map, cmap='hot')
axes[0, 1].set_title(f'Mapa de Anomalías ({n_anomalies} detectadas)', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# Imagen con anomalías marcadas
img_marked = img.copy()
axes[1, 0].imshow(img_marked, cmap='gray')
axes[1, 0].scatter(positions[predictions == -1, 1], 
                   positions[predictions == -1, 0], 
                   c='red', s=50, marker='x', linewidths=2)
axes[1, 0].set_title('Anomalías Marcadas en la Imagen', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# Visualización t-SNE de características
print("\n[1.3] Visualizando características con t-SNE...")
if len(features) > 50:
    tsne = TSNE(n_components=2, perplexity=min(30, len(features)//3))
    features_2d = tsne.fit_transform(features)
    
    axes[1, 1].scatter(features_2d[predictions == 1, 0], 
                      features_2d[predictions == 1, 1], 
                      c='blue', alpha=0.6, s=20, label='Normal')
    axes[1, 1].scatter(features_2d[predictions == -1, 0], 
                      features_2d[predictions == -1, 1], 
                      c='red', alpha=0.8, s=40, marker='x', label='Anomalía')
    axes[1, 1].set_title('Visualización t-SNE de Características', fontsize=12, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Insuficientes datos\npara t-SNE', 
                   ha='center', va='center', fontsize=14)
    axes[1, 1].axis('off')

plt.suptitle('Detección de Anomalías en Imágenes', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Ejemplo 1 completado")

# ==============================================================================
# EJEMPLO 2: DETECCIÓN DE PUNTOS DE CAMBIO EN PERFILES DE IMAGEN
# ==============================================================================

print("\n" + "="*80)
print("EJEMPLO 2: Detección de Puntos de Cambio en Perfiles de Imagen")
print("="*80)

def detect_changepoints_1d(signal_1d, max_changes=5):
    """Detecta puntos de cambio en un perfil 1D"""
    n = len(signal_1d)
    window_size = max(10, n // 50)
    
    # Calcular RMS en ventanas
    rms_values = []
    for i in range(n - window_size):
        window = signal_1d[i:i+window_size]
        rms_values.append(np.sqrt(np.mean(window**2)))
    rms_values = np.array(rms_values)
    
    # Detectar cambios
    diff = np.abs(np.diff(rms_values))
    threshold = np.mean(diff) + 2*np.std(diff)
    candidates = np.where(diff > threshold)[0]
    
    if len(candidates) > max_changes:
        magnitudes = diff[candidates]
        top_indices = np.argsort(magnitudes)[-max_changes:]
        candidates = candidates[top_indices]
        candidates = np.sort(candidates)
    
    return candidates

print("\n[2.1] Analizando perfiles horizontales y verticales...")

# Perfil horizontal (promedio por fila)
horizontal_profile = np.mean(img, axis=1)
changepoints_h = detect_changepoints_1d(horizontal_profile, max_changes=5)

# Perfil vertical (promedio por columna)
vertical_profile = np.mean(img, axis=0)
changepoints_v = detect_changepoints_1d(vertical_profile, max_changes=5)

print(f"✓ Puntos de cambio horizontales: {len(changepoints_h)}")
print(f"✓ Puntos de cambio verticales: {len(changepoints_v)}")

# Visualización
fig = plt.figure(figsize=(15, 10))

# Imagen con líneas de cambio
ax1 = plt.subplot(2, 2, 1)
ax1.imshow(img, cmap='gray')
for cp in changepoints_h:
    ax1.axhline(y=cp, color='r', linestyle='--', linewidth=2, alpha=0.7)
for cp in changepoints_v:
    ax1.axvline(x=cp, color='b', linestyle='--', linewidth=2, alpha=0.7)
ax1.set_title('Puntos de Cambio en la Imagen', fontsize=12, fontweight='bold')
ax1.axis('off')

# Perfil horizontal
ax2 = plt.subplot(2, 2, 2)
ax2.plot(horizontal_profile, np.arange(len(horizontal_profile)), 'k-', linewidth=1.5)
ax2.plot(horizontal_profile[changepoints_h], changepoints_h, 'ro', 
         markersize=10, markerfacecolor='r', label='Puntos de Cambio')
ax2.set_xlabel('Intensidad Promedio')
ax2.set_ylabel('Fila')
ax2.set_title('Perfil Horizontal', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

# Perfil vertical
ax3 = plt.subplot(2, 2, 3)
ax3.plot(vertical_profile, 'k-', linewidth=1.5)
ax3.plot(changepoints_v, vertical_profile[changepoints_v], 'bo', 
         markersize=10, markerfacecolor='b', label='Puntos de Cambio')
ax3.set_xlabel('Columna')
ax3.set_ylabel('Intensidad Promedio')
ax3.set_title('Perfil Vertical', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Perfil diagonal
diagonal_profile = np.diag(img)
changepoints_d = detect_changepoints_1d(diagonal_profile, max_changes=5)

ax4 = plt.subplot(2, 2, 4)
ax4.plot(diagonal_profile, 'k-', linewidth=1.5)
ax4.plot(changepoints_d, diagonal_profile[changepoints_d], 'go', 
         markersize=10, markerfacecolor='g', label='Puntos de Cambio')
ax4.set_xlabel('Posición Diagonal')
ax4.set_ylabel('Intensidad')
ax4.set_title('Perfil Diagonal', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('Detección de Puntos de Cambio en Perfiles de Imagen', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Ejemplo 2 completado")

# ==============================================================================
# EJEMPLO 3: GRÁFICOS DE CONTROL DE CALIDAD EN IMÁGENES
# ==============================================================================

print("\n" + "="*80)
print("EJEMPLO 3: Gráficos de Control de Calidad en Imágenes")
print("="*80)

print("\n[3.1] Analizando estadísticas de bloques de la imagen...")

def image_quality_control(image, block_size=32):
    """
    Crea gráfico de control basado en estadísticas de bloques.
    
    Parámetros:
    - image: Imagen de entrada
    - block_size: Tamaño del bloque
    
    Retorna:
    - stats: Estadísticas de cada bloque
    - violations: Índices de bloques con violaciones
    """
    h, w = image.shape
    block_means = []
    block_stds = []
    block_positions = []
    
    for i in range(0, h - block_size, block_size):
        for j in range(0, w - block_size, block_size):
            block = image[i:i+block_size, j:j+block_size]
            block_means.append(np.mean(block))
            block_stds.append(np.std(block))
            block_positions.append((i, j))
    
    block_means = np.array(block_means)
    block_stds = np.array(block_stds)
    
    # Calcular límites de control
    cl_mean = np.mean(block_means)
    sigma_mean = np.std(block_means)
    
    ucl = cl_mean + 3*sigma_mean
    lcl = cl_mean - 3*sigma_mean
    upper_2sigma = cl_mean + 2*sigma_mean
    lower_2sigma = cl_mean - 2*sigma_mean
    
    # Detectar violaciones (regla WE2)
    violations = []
    for i in range(len(block_means) - 2):
        window = block_means[i:i+3]
        count_upper = np.sum(window > upper_2sigma)
        count_lower = np.sum(window < lower_2sigma)
        
        if count_upper >= 2 or count_lower >= 2:
            violations.extend([i, i+1, i+2])
    
    violations = list(set(violations))
    
    stats = {
        'means': block_means,
        'stds': block_stds,
        'positions': block_positions,
        'cl': cl_mean,
        'ucl': ucl,
        'lcl': lcl,
        'upper_2sigma': upper_2sigma,
        'lower_2sigma': lower_2sigma
    }
    
    return stats, violations

stats, violations = image_quality_control(img, block_size=32)

print(f"✓ Bloques analizados: {len(stats['means'])}")
print(f"✓ Línea central: {stats['cl']:.4f}")
print(f"✓ UCL: {stats['ucl']:.4f}, LCL: {stats['lcl']:.4f}")
print(f"✓ Violaciones detectadas: {len(violations)}")

# Crear mapa de calidad
quality_map = np.ones_like(img) * stats['cl']
for idx, (i, j) in enumerate(stats['positions']):
    quality_map[i:i+32, j:j+32] = stats['means'][idx]

violation_map = np.zeros_like(img)
for idx in violations:
    if idx < len(stats['positions']):
        i, j = stats['positions'][idx]
        violation_map[i:i+32, j:j+32] = 1

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Imagen original
axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Imagen Original', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Mapa de calidad
im1 = axes[0, 1].imshow(quality_map, cmap='jet')
axes[0, 1].set_title('Mapa de Intensidad Media por Bloque', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')
plt.colorbar(im1, ax=axes[0, 1])

# Gráfico de control
axes[1, 0].plot(stats['means'], 'bo-', linewidth=1.5, markersize=4, label='Media de bloques')
axes[1, 0].axhline(y=stats['cl'], color='g', linestyle='-', linewidth=2, label='Línea Central')
axes[1, 0].axhline(y=stats['ucl'], color='r', linestyle='--', linewidth=1.5, label='Límite 3σ')
axes[1, 0].axhline(y=stats['lcl'], color='r', linestyle='--', linewidth=1.5)
axes[1, 0].axhline(y=stats['upper_2sigma'], color='m', linestyle='--', linewidth=1.5, label='Límite 2σ')
axes[1, 0].axhline(y=stats['lower_2sigma'], color='m', linestyle='--', linewidth=1.5)

if violations:
    axes[1, 0].plot(violations, stats['means'][violations], 'rx', 
                   markersize=12, markeredgewidth=3, label='Violaciones WE2')

axes[1, 0].set_xlabel('Número de Bloque')
axes[1, 0].set_ylabel('Intensidad Media')
axes[1, 0].set_title('Gráfico de Control de Calidad', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='best', fontsize=8)
axes[1, 0].grid(True, alpha=0.3)

# Mapa de violaciones
axes[1, 1].imshow(img, cmap='gray', alpha=0.7)
axes[1, 1].imshow(violation_map, cmap='Reds', alpha=0.5)
axes[1, 1].set_title(f'Bloques con Violaciones ({len(violations)})', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Control de Calidad en Imágenes (Regla WE2)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Ejemplo 3 completado")

# ==============================================================================
# EJEMPLO 4: FILTRADO DE MEDIANA EN IMÁGENES 2D
# ==============================================================================

print("\n" + "="*80)
print("EJEMPLO 4: Filtrado de Mediana en Imágenes 2D")
print("="*80)

print("\n[4.1] Añadiendo diferentes tipos de ruido...")

# Añadir ruido sal y pimienta
def add_salt_pepper(image, salt_prob=0.02, pepper_prob=0.02):
    noisy = image.copy()
    salt_mask = np.random.random(image.shape) < salt_prob
    noisy[salt_mask] = 1
    pepper_mask = np.random.random(image.shape) < pepper_prob
    noisy[pepper_mask] = 0
    return noisy

# Añadir ruido gaussiano
def add_gaussian(image, sigma=0.1):
    noisy = image + np.random.randn(*image.shape) * sigma
    return np.clip(noisy, 0, 1)

img_sp = add_salt_pepper(img, salt_prob=0.03, pepper_prob=0.03)
img_gaussian = add_gaussian(img, sigma=0.1)
img_mixed = add_salt_pepper(add_gaussian(img, sigma=0.05), salt_prob=0.02, pepper_prob=0.02)

print("✓ Ruido sal y pimienta añadido (6%)")
print("✓ Ruido gaussiano añadido (σ=0.1)")
print("✓ Ruido mixto añadido")

print("\n[4.2] Aplicando filtros de mediana con diferentes tamaños...")

# Aplicar filtros de mediana
kernel_sizes = [3, 5, 7, 9]
filtered_images = {}

for k in kernel_sizes:
    filtered_images[f'sp_{k}'] = signal.medfilt2d(img_sp, kernel_size=k)
    filtered_images[f'gaussian_{k}'] = signal.medfilt2d(img_gaussian, kernel_size=k)
    filtered_images[f'mixed_{k}'] = signal.medfilt2d(img_mixed, kernel_size=k)

print(f"✓ Filtros aplicados: {kernel_sizes}")

# Visualización 1: Comparación de ruido sal y pimienta
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray', vmin=0, vmax=1)
axes[0, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_sp, cmap='gray', vmin=0, vmax=1)
axes[0, 1].set_title('Con Ruido Sal y Pimienta (6%)', fontsize=11, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(filtered_images['sp_3'], cmap='gray', vmin=0, vmax=1)
axes[0, 2].set_title('Filtrada (3x3)', fontsize=11, fontweight='bold')
axes[0, 2].axis('off')

axes[1, 0].imshow(filtered_images['sp_5'], cmap='gray', vmin=0, vmax=1)
axes[1, 0].set_title('Filtrada (5x5)', fontsize=11, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(filtered_images['sp_7'], cmap='gray', vmin=0, vmax=1)
axes[1, 1].set_title('Filtrada (7x7)', fontsize=11, fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(filtered_images['sp_9'], cmap='gray', vmin=0, vmax=1)
axes[1, 2].set_title('Filtrada (9x9)', fontsize=11, fontweight='bold')
axes[1, 2].axis('off')

plt.suptitle('Filtrado de Mediana - Ruido Sal y Pimienta', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Visualización 2: Comparación de diferentes tipos de ruido
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Fila 1: Sal y Pimienta
axes[0, 0].imshow(img, cmap='gray', vmin=0, vmax=1)
axes[0, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_sp, cmap='gray', vmin=0, vmax=1)
axes[0, 1].set_title('Sal y Pimienta', fontsize=11, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(filtered_images['sp_5'], cmap='gray', vmin=0, vmax=1)
axes[0, 2].set_title('Filtrada (5x5)', fontsize=11, fontweight='bold')
axes[0, 2].axis('off')

# Fila 2: Gaussiano
axes[1, 0].imshow(img, cmap='gray', vmin=0, vmax=1)
axes[1, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gaussian, cmap='gray', vmin=0, vmax=1)
axes[1, 1].set_title('Gaussiano (σ=0.1)', fontsize=11, fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(filtered_images['gaussian_5'], cmap='gray', vmin=0, vmax=1)
axes[1, 2].set_title('Filtrada (5x5)', fontsize=11, fontweight='bold')
axes[1, 2].axis('off')

# Fila 3: Mixto
axes[2, 0].imshow(img, cmap='gray', vmin=0, vmax=1)
axes[2, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[2, 0].axis('off')

axes[2, 1].imshow(img_mixed, cmap='gray', vmin=0, vmax=1)
axes[2, 1].set_title('Ruido Mixto', fontsize=11, fontweight='bold')
axes[2, 1].axis('off')

axes[2, 2].imshow(filtered_images['mixed_5'], cmap='gray', vmin=0, vmax=1)
axes[2, 2].set_title('Filtrada (5x5)', fontsize=11, fontweight='bold')
axes[2, 2].axis('off')

plt.suptitle('Filtrado de Mediana - Comparación de Tipos de Ruido', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Calcular métricas de calidad
def calculate_psnr(original, filtered):
    mse = np.mean((original - filtered)**2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(1.0 / np.sqrt(mse))

print("\n[4.3] Métricas de calidad (PSNR):")
for k in kernel_sizes:
    psnr_sp = calculate_psnr(img, filtered_images[f'sp_{k}'])
    psnr_gaussian = calculate_psnr(img, filtered_images[f'gaussian_{k}'])
    psnr_mixed = calculate_psnr(img, filtered_images[f'mixed_{k}'])
    
    print(f"\nKernel {k}x{k}:")
    print(f"  Sal y Pimienta: {psnr_sp:.2f} dB")
    print(f"  Gaussiano: {psnr_gaussian:.2f} dB")
    print(f"  Mixto: {psnr_mixed:.2f} dB")

print("\n✓ Ejemplo 4 completado")

# ==============================================================================
# RESUMEN FINAL
# ==============================================================================

print("\n" + "="*80)
print("TODOS LOS EJEMPLOS COMPLETADOS")
print("="*80)

print("""
Resumen de análisis realizados sobre la imagen:

1. Detección de Anomalías ✓
   - Extracción de características locales de parches
   - Clasificación con One-Class SVM
   - Visualización con t-SNE
   - Mapa de anomalías generado

2. Detección de Puntos de Cambio ✓
   - Análisis de perfiles horizontales, verticales y diagonales
   - Detección de cambios estructurales
   - Marcado de líneas de cambio en la imagen

3. Gráficos de Control de Calidad ✓
   - Análisis estadístico por bloques
   - Aplicación de regla Western Electric WE2
   - Detección de bloques fuera de control
   - Mapa de calidad generado

4. Filtrado de Mediana ✓
   - Comparación de diferentes tamaños de kernel (3x3, 5x5, 7x7, 9x9)
   - Aplicado a tres tipos de ruido:
     * Sal y Pimienta
     * Gaussiano
     * Mixto
   - Métricas PSNR calculadas

Todas las visualizaciones han sido generadas correctamente.
""")

print("="*80)